In [ ]:
!pip install datasets

In [235]:
import pandas as pd
import numpy as np
import os
from numpy import array
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from datasets import load_dataset, concatenate_datasets, load_from_disk, Dataset

In [236]:
# Get Annotations based on Text
def get_annotations_de(annotations):
  def search(annotation, annotation_context):
      for split in ['train', 'validation', 'test']:
          for i, example in enumerate(dataset[split]):
              if example['translation']['en'] == annotation and \
                  dataset[split][i-1]['translation']['en'] == annotation_context[0] and \
                   dataset[split][i-2]['translation']['en'] == annotation_context[1] and \
                    dataset[split][i-3]['translation']['en'] == annotation_context[2]:
                  return example
      return None

  result = []
  for index, row in annotations.iterrows():
      annotation_en = eval(row[2])["en"]
      annotation_context = eval(row[3])["en"]
      match = search(annotation_en, np.flip(annotation_context))
      if match == None:
        match = {'doc_id': row[0], 'seg_id': row[1], 'translation': {'en': eval(row[2])['en'], 'de': None}, 'context': {'en': annotation_context, 'de': None}}
      else:
        match['context'] = eval(row[3])
      result.append(match)

  return result

In [237]:
def translate_to_de(sentence):
  model_inputs = tokenizer(sentence, return_tensors="pt")

  generated_tokens = model.generate(
      **model_inputs,
      forced_bos_token_id=tokenizer.lang_code_to_id["de_DE"]
  )
  return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

In [238]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

In [239]:
dataset = load_dataset("gsarti/iwslt2017_context", "iwslt2017-en-de")

In [240]:
annotations_b1 = pd.read_csv("annotations/annotations_bjurn_b1_10.csv")
annotations_b2 = pd.read_csv("annotations/annotations_bjurn_b2_10.csv")

In [241]:
annotations_de_b1 = get_annotations_de(annotations_b1)
annotations_de_b2 = get_annotations_de(annotations_b2)

In [242]:
for i, item in enumerate(annotations_de_b1):
  print(f"Progress: {i+1}/{len(annotations_de_b1)}")
  if item['translation']['de'] is None:
    de_translation = translate_to_de(item['translation']['en'])
    context = []
    for context_item in item['context']['en']:
      de_translation_context = translate_to_de(context_item)
      context.append(de_translation_context)

    item['translation']['de'] = de_translation
    item['context']['de'] = context
  else:
    context = []
    for context_item in item['context']['en']:
      de_translation_context = translate_to_de(context_item)
      context.append(de_translation_context)
    item['context']['de'] = context
    del(item['context']['nl'])

Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


In [ ]:
annotations_de_b1

In [243]:
for i, item in enumerate(annotations_de_b2):
  print(f"Progress: {i+1}/{len(annotations_de_b2)}")
  if item['translation']['de'] is None:
    de_translation = translate_to_de(item['translation']['en'])
    context = []
    for context_item in item['context']['en']:
      de_translation_context = translate_to_de(context_item)
      context.append(de_translation_context)

    item['translation']['de'] = de_translation[0]
    item['context']['de'] = context
  else:
    context = []
    for context_item in item['context']['en']:
      de_translation_context = translate_to_de(context_item)
      context.append(de_translation_context)
    item['context']['de'] = context
    del(item['context']['nl'])

Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


In [244]:
annotations_de = annotations_de_b1 + annotations_de_b2

In [245]:
annotations_de

[{'doc_id': 103,
  'seg_id': 49,
  'translation': {'en': 'And I said, "Well, how do you hear it?"',
   'de': 'Und ich sagte: "Nun, wie hörst du es?"'},
  'context': {'en': array(['And so on. There is a little bit of a difference there that is worth just --  -- thinking about. And I remember when I was 12 years old, and I started playing tympani and percussion, and my teacher said, "Well, how are we going to do this? You know, music is about listening."',
          'And I said, "Yes, I agree with that. So what\'s the problem?"',
          'And he said, "Well, how are you going to hear this? How are you going to hear that?"'],
         dtype=object),
   'de': ['Und so weiter. Da gibt es ein wenig Unterschied, den es wert ist - - darüber nachzudenken. Und ich erinnere mich, als ich 12 Jahre alt war, und ich begann Tympani und Percussion zu spielen, und mein Lehrer sagte, "Nun, wie werden wir das tun? Du weißt, Musik geht darum, zu hören."',
    'Und ich sagte: "Ja, ich bin damit einversta

In [247]:
data_dict = {key: [dic.get(key) for dic in annotations_de] for key in annotations_de[0]}

dataset = Dataset.from_dict(data_dict)

dataset.save_to_disk('annotations_de/')
dataset.to_csv('annotations_de.csv')

Saving the dataset (0/1 shards):   0%|          | 0/20 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

15193